#  Projet NLP : Reconnaissance des Entités Nommées (NER) en langue française


## 1. Contexte et motivation du projet

La croissance exponentielle des données textuelles issues des rapports, articles, réseaux sociaux et documents institutionnels pose un défi majeur en matière d’extraction automatique d’informations pertinentes. Le traitement manuel de ces données est coûteux, chronophage et sujet à des erreurs humaines.

Dans ce contexte, la Reconnaissance des Entités Nommées (Named Entity Recognition – NER) constitue une tâche fondamentale du Traitement Automatique du Langage Naturel (NLP). Elle vise à identifier et à classifier automatiquement, au sein d’un texte, des entités sémantiques telles que les noms de personnes, de lieux, d’organisations, de produits, de dates ou de montants.

Ce projet s’inscrit dans cette dynamique et vise à concevoir un système automatisé de NER appliqué à des textes en langue française, en s’appuyant sur des données annotées de référence et des modèles modernes.


## 2. Objectifs du projet

Les objectifs poursuivis dans ce projet sont les suivants :

- Construire un modèle capable d’extraire et de classifier automatiquement les entités nommées présentes dans un texte
- Exploiter un corpus annoté riche et complexe afin de couvrir une large diversité d’entités
- Comparer différentes approches de modélisation et de vectorisation
- Évaluer la performance du modèle à l’aide de métriques adaptées (précision, rappel, F1-score)
- Déployer un outil de prédiction sous forme d’API ou d’interface interactive


## 3. Présentation du jeu de données

### 3.1 Description générale

Le projet repose sur un corpus annoté au format CoNLL, issu du dataset MultiCoNER v2 pour la langue française. Ce corpus se distingue par la richesse et la diversité de ses annotations, incluant des entités fines, ambiguës et dépendantes du contexte.

Les annotations suivent le schéma standard BIO :
- B-XXX : début d’une entité nommée
- I-XXX : continuation de l’entité
- O : absence d’entité


### 3.2 Organisation des données

Les données sont organisées selon une partition classique en apprentissage automatique :

- Ensemble d’entraînement (train) : utilisé pour l’apprentissage du modèle
- Ensemble de validation (dev) : utilisé pour l’ajustement des hyperparamètres
- Ensemble de test (test) : utilisé pour l’évaluation finale des performances

Chaque ensemble est stocké dans un fichier distinct au format .conll.


## 4. Chargement des données

> Les fichiers CoNLL sont déjà disponibles localement dans le dossier `data/` du repository. On utilise donc des chemins relatifs pour garantir la reproductibilité.

In [2]:
from pathlib import Path

DATA_DIR = Path("..") / "data"
train_path = DATA_DIR / "fr_train.conll"
dev_path   = DATA_DIR / "fr_dev.conll"
test_path  = DATA_DIR / "fr_test.conll"

for p in [train_path, dev_path, test_path]:
    print(p, "->", p.exists())

..\data\fr_train.conll -> True
..\data\fr_dev.conll -> True
..\data\fr_test.conll -> True


## 5. Lecture et structuration des fichiers CoNLL

Afin de rendre les fichiers CoNLL exploitables par les modèles de traitement automatique du langage, une fonction de lecture est définie pour reconstruire les phrases et associer chaque token à son étiquette correspondante.


In [3]:
def read_conll(path):
    """Lecture d'un fichier CoNLL (BIO)."""
    sentences = []
    labels = []

    sentence = []
    sentence_labels = []

    with open(path, encoding="utf-8") as f:
        for line in f:
            line = line.strip()

            # fin de phrase
            if not line:
                if sentence:
                    sentences.append(sentence)
                    labels.append(sentence_labels)
                    sentence = []
                    sentence_labels = []
                continue

            parts = line.split()
            token = parts[0]
            ner_tag = parts[-1]

            sentence.append(token)
            sentence_labels.append(ner_tag)

    # flush final
    if sentence:
        sentences.append(sentence)
        labels.append(sentence_labels)

    return sentences, labels

## 6. Chargement des ensembles d’entraînement, de validation et de test

Les fichiers sont ensuite chargés en mémoire et séparés selon leur rôle dans le processus d’apprentissage et d’évaluation.


In [4]:
data_train = train_path
data_dev   = dev_path
data_test  = test_path

train_sent, train_labels = read_conll(data_train)
dev_sent, dev_labels     = read_conll(data_dev)
test_sent, test_labels   = read_conll(data_test)

## 7. Vérification de l’intégrité des données

Avant de procéder à la phase de modélisation, une vérification est effectuée afin de s’assurer de la cohérence des données chargées et de la bonne reconstruction des phrases et des annotations.


In [5]:
print("Train:", len(train_sent))
print("Dev  :", len(dev_sent))
print("Test :", len(test_sent))

print(train_sent[0])
print(train_labels[0])

Train: 16548
Dev  : 857
Test : 249786
['#', 'elle', 'porte', 'le', 'nom', 'de', 'la', 'romancière', 'américaine', 'susan', 'sontag', '(', '1933', '2004', ')', '.']
['domain=fr', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Artist', 'I-Artist', 'O', 'O', 'O', 'O', 'O']


## 8. Modélisation et évaluation

> Les étapes suivantes couvrent l'analyse exploratoire, deux modèles (baseline et CRF), l'évaluation et la sauvegarde du modèle pour l'API.

### 8.1 Analyse exploratoire des labels

> Distribution des étiquettes BIO dans l'ensemble d'entraînement.

In [6]:
from collections import Counter

label_counts = Counter([l for labs in train_labels for l in labs])
print("Nb labels:", len(label_counts))
print("Top 10 labels:", label_counts.most_common(10))

unique_labels = sorted(label_counts.keys())
print("Exemple labels:", unique_labels[:20])

Nb labels: 68
Top 10 labels: [('O', 196008), ('domain=fr', 16548), ('I-Artist', 3787), ('B-Artist', 3634), ('I-VisualWork', 3608), ('B-HumanSettlement', 2932), ('B-WrittenWork', 2268), ('I-OtherPER', 2176), ('B-OtherPER', 1874), ('B-VisualWork', 1794)]
Exemple labels: ['B-AerospaceManufacturer', 'B-AnatomicalStructure', 'B-ArtWork', 'B-Artist', 'B-Athlete', 'B-CarManufacturer', 'B-Cleric', 'B-Clothing', 'B-Disease', 'B-Drink', 'B-Facility', 'B-Food', 'B-HumanSettlement', 'B-MedicalProcedure', 'B-Medication/Vaccine', 'B-MusicalGRP', 'B-MusicalWork', 'B-ORG', 'B-OtherLOC', 'B-OtherPER']


### 8.2 Baseline : Logistic Regression (token-level)

> Modèle simple pour établir un point de comparaison.

In [7]:
import sys
import time
from pathlib import Path

sys.path.append(str(Path("..").resolve()))

from src.features import sent2features

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from seqeval.metrics import classification_report, f1_score

def predict_logreg(model, sentences):
    all_feats = [feat for sent in sentences for feat in sent2features(sent)]
    all_preds = model.predict(all_feats)

    preds = []
    idx = 0
    for sent in sentences:
        preds.append(list(all_preds[idx : idx + len(sent)]))
        idx += len(sent)
    return preds

# Stats du dataset
print("=" * 60)
print("DATASET STATISTICS")
print("=" * 60)
total_tokens_train = sum(len(s) for s in train_sent)
total_tokens_dev = sum(len(s) for s in dev_sent)
print(f"Train: {len(train_sent):,} phrases, {total_tokens_train:,} tokens")
print(f"Dev:   {len(dev_sent):,} phrases, {total_tokens_dev:,} tokens")
print("=" * 60)

print("\n[1/3] Extracting features...")
start = time.time()
X_train = [feat for sent in train_sent for feat in sent2features(sent)]
y_train = [lab for labs in train_labels for lab in labs]
feat_time = time.time() - start
print(f"✓ Features extracted in {feat_time:.2f}s ({len(X_train):,} samples)")

print("\n[2/3] Training Logistic Regression...")
print("     (Using: solver='saga', max_iter=100, tol=1e-2 for speed)")
start = time.time()

baseline = Pipeline(
    [
        ("vec", DictVectorizer(sparse=True)),
        (
            "clf",
            LogisticRegression(
                solver="saga",
                max_iter=100,
                tol=1e-2,
                n_jobs=-1,
                class_weight="balanced",
                random_state=42,
            ),
        ),
    ]
)

baseline.fit(X_train, y_train)
train_time = time.time() - start
print(f"✓ Model trained in {train_time:.2f}s")

print("\n[3/3] Predicting and evaluating...")
start = time.time()
preds_dev = predict_logreg(baseline, dev_sent)
pred_time = time.time() - start
print(f"✓ Predictions made in {pred_time:.2f}s")

print("\n" + "=" * 60)
print("RESULTS")
print("=" * 60)
print("F1 (dev):", f1_score(dev_labels, preds_dev))
print(classification_report(dev_labels, preds_dev))
print("=" * 60)


DATASET STATISTICS
Train: 16,548 phrases, 264,291 tokens
Dev:   857 phrases, 13,919 tokens

[1/3] Extracting features...
✓ Features extracted in 12.62s (264,291 samples)

[2/3] Training Logistic Regression...
     (Using: solver='saga', max_iter=100, tol=1e-2 for speed)


c:\Users\hp\OneDrive\Bureau\NLP\ner-multiconer\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
c:\Users\hp\OneDrive\Bureau\NLP\ner-multiconer\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


✓ Model trained in 681.92s

[3/3] Predicting and evaluating...
✓ Predictions made in 4.38s

RESULTS


c:\Users\hp\OneDrive\Bureau\NLP\ner-multiconer\.venv\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: domain=fr seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


F1 (dev): 0.22498667140572243
                       precision    recall  f1-score   support

AerospaceManufacturer       0.12      0.91      0.21        11
  AnatomicalStructure       0.06      0.47      0.10        15
              ArtWork       0.01      0.23      0.02        13
               Artist       0.05      0.11      0.07       189
              Athlete       0.10      0.29      0.15        72
      CarManufacturer       0.09      0.54      0.15        13
               Cleric       0.09      0.30      0.14        20
             Clothing       0.01      0.27      0.02        11
              Disease       0.02      0.44      0.04        16
                Drink       0.04      0.36      0.07        11
             Facility       0.02      0.22      0.03        54
                 Food       0.01      0.38      0.02        13
      HumanSettlement       0.24      0.48      0.32       155
     MedicalProcedure       0.01      0.45      0.02        11
   Medication/Vaccine   

### 8.3 Modèle séquentiel : CRF

> Le CRF capture les dépendances entre étiquettes successives.

In [8]:
from sklearn_crfsuite import CRF

X_train_seq = [sent2features(sent) for sent in train_sent]
X_dev_seq   = [sent2features(sent) for sent in dev_sent]

crf = CRF(
    algorithm="lbfgs",
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
 )

crf.fit(X_train_seq, train_labels)

preds_dev = crf.predict(X_dev_seq)
print("F1 (dev):", f1_score(dev_labels, preds_dev))
print(classification_report(dev_labels, preds_dev))

F1 (dev): 0.7211281303185024
                       precision    recall  f1-score   support

AerospaceManufacturer       0.78      0.64      0.70        11
  AnatomicalStructure       0.86      0.40      0.55        15
              ArtWork       0.67      0.15      0.25        13
               Artist       0.55      0.60      0.57       189
              Athlete       0.39      0.42      0.41        72
      CarManufacturer       0.80      0.62      0.70        13
               Cleric       0.50      0.30      0.37        20
             Clothing       1.00      0.27      0.43        11
              Disease       0.57      0.25      0.35        16
                Drink       0.67      0.36      0.47        11
             Facility       0.71      0.56      0.63        54
                 Food       0.75      0.46      0.57        13
      HumanSettlement       0.76      0.64      0.69       155
     MedicalProcedure       0.83      0.45      0.59        11
   Medication/Vaccine    

### 8.4 Sauvegarde du modèle (pour l'API)

> Le modèle sauvegardé sera utilisé par l'API FastAPI.

In [9]:
from pathlib import Path
import joblib

MODEL_DIR = Path("..") / "models"
MODEL_DIR.mkdir(parents=True, exist_ok=True)

joblib.dump(crf, MODEL_DIR / "ner_model.joblib")
print("Model saved to", MODEL_DIR / "ner_model.joblib")

Model saved to ..\models\ner_model.joblib


### 8.5 Évaluation sur test

> Performance finale sur l'ensemble de test.

In [1]:
X_test = [sent2features(sent) for sent in test_sent]
preds_test = crf.predict(X_test)

print("=== TEST SET ===")
print("F1:", f1_score(test_labels, preds_test))
print(classification_report(test_labels, preds_test))

NameError: name 'test_sent' is not defined